# Cash Balance of Loans

In [ ]:
import pandas as pd
import numpy as np
import time
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inlineN
plt.style.use("ggplot")
import seaborn as sns

import pylab
pylab.rcParams['figure.figsize'] = (15, 10)

In [ ]:
balance = pd.read_csv("POS_CASH_balance.csv")
# app_train = pd.read_csv("app_train_red.csv", index_col=0)
prev_apps = pd.read_csv("previous_application.csv")
pmts = pd.read_csv("installments_payments.csv")

In [ ]:
balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


# Notes
- Can use dataset to determine how many days late the payments were, but should also be able to do the from the data containing the payments since it has the "date" the installment was supposed to be made and the "date" the payments were actually made
    - trying to see how they relate to each other
- Can definitely check to see if they have an open loan or finished paying the loan.

In [ ]:
ID = 2448283
prev_apps[prev_apps.SK_ID_PREV == ID]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
282244,2448283,226558,Consumer loans,3880.305,20385.0,19030.5,2250.0,20385.0,SATURDAY,15,...,Connectivity,6.0,high,POS mobile with interest,365243.0,-1593.0,-1443.0,-1443.0,-577.0,0.0


In [ ]:
balance[balance.SK_ID_PREV == ID].sort_values("MONTHS_BALANCE")

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1439944,2448283,226558,-53,6.0,6.0,Active,0,0
3290550,2448283,226558,-52,6.0,5.0,Active,0,0
9476097,2448283,226558,-51,6.0,4.0,Active,0,0
5952719,2448283,226558,-50,6.0,3.0,Active,0,0
5916746,2448283,226558,-49,6.0,2.0,Active,0,0
7696039,2448283,226558,-48,6.0,1.0,Active,0,0
5716028,2448283,226558,-47,6.0,0.0,Active,20,0
1782433,2448283,226558,-46,6.0,0.0,Active,50,0
7464551,2448283,226558,-45,6.0,0.0,Active,81,0
5744905,2448283,226558,-44,6.0,0.0,Active,112,0


In [ ]:
pmts[pmts.SK_ID_PREV == ID].sort_values("NUM_INSTALMENT_NUMBER")

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
7488149,2448283,226558,1.0,1,-1593.0,-1603.0,3880.305,3880.305
6139045,2448283,226558,1.0,2,-1563.0,-1570.0,3880.305,3880.305
5710890,2448283,226558,1.0,3,-1533.0,-1535.0,3880.305,3880.305
4831186,2448283,226558,1.0,4,-1503.0,-1503.0,3880.305,3880.305
7362866,2448283,226558,1.0,5,-1473.0,-1472.0,3880.305,3880.305
6627705,2448283,226558,1.0,6,-1443.0,-1442.0,3885.660,3880.305
7130666,2448283,226558,1.0,6,-1443.0,-581.0,3885.660,5.355


In [ ]:
(True | False)

True

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9263f292-ea73-48e7-8892-f3d3ed1db03c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>